In [1]:
from common import Common
from pyspark import *
from pyspark.streaming import *
from pyspark import SparkContext, SparkConf

In [2]:
common = Common()
sc = common.get_sc()

<SparkContext master=local appName=myapp>
2.4.5


## Question 1

In [4]:
file = '/home/ec2-user/data/blogtexts'
rdd = sc.textFile(file)
print ('Count: ', rdd.count())
print ('First: ', rdd.first())

Count:  484
First:  Think of it for a moment – 1 Qunitillion = 1 Million Billion! Can you imagine how many drives / CDs / Blue-ray DVDs would be required to store them? It is difficult to imagine this scale of data generation even as a data science professional. While this pace of data generation is very exciting,  it has created entirely new set of challenges and has forced us to find new ways to handle Big Huge data effectively.


In [6]:
def lowercase(lines):
    lines = lines.lower()
    lines = lines.split()
    return lines
rdd1 = rdd.map(lowercase)
print ('Count: ', rdd1.count())
print ('First: ', rdd1.first())

Count:  484
First:  ['think', 'of', 'it', 'for', 'a', 'moment', '–', '1', 'qunitillion', '=', '1', 'million', 'billion!', 'can', 'you', 'imagine', 'how', 'many', 'drives', '/', 'cds', '/', 'blue-ray', 'dvds', 'would', 'be', 'required', 'to', 'store', 'them?', 'it', 'is', 'difficult', 'to', 'imagine', 'this', 'scale', 'of', 'data', 'generation', 'even', 'as', 'a', 'data', 'science', 'professional.', 'while', 'this', 'pace', 'of', 'data', 'generation', 'is', 'very', 'exciting,', 'it', 'has', 'created', 'entirely', 'new', 'set', 'of', 'challenges', 'and', 'has', 'forced', 'us', 'to', 'find', 'new', 'ways', 'to', 'handle', 'big', 'huge', 'data', 'effectively.']


## Question 2

In [10]:
rdd2 = rdd1.flatMap(lambda x:x)
print ('Count: ', rdd2.count())
print ('First 5: ', rdd2.take(5))

Count:  5304
First 5:  ['think', 'of', 'it', 'for', 'a']


## Question 3

In [14]:
import re

stopwords = ['is','am','are','the','for','a']
rdd3 = rdd2.filter(lambda x: x not in stopwords).filter(lambda x : re.match("^[A-Za-z0-9]*$", x))
print ('Count: ', rdd3.count())
print ('First 5: ', rdd3.take(5))

Count:  3860
First 5:  ['think', 'of', 'it', 'moment', '1']


## Question 4

In [26]:
rdd4 = rdd3.groupBy(lambda w: w[0:3])
print ('Count: ', rdd4.count())
print ('First 5: ', rdd4.take(5))
print ('\nFirst 3 letter group:', rdd4.first()[0], list(rdd4.first()[1]))
# print ('\tTokens', list(rdd4.first()[1]))

Count:  467
First 5:  [('thi', <pyspark.resultiterable.ResultIterable object at 0x7f73a962a190>), ('of', <pyspark.resultiterable.ResultIterable object at 0x7f73a962ab50>), ('it', <pyspark.resultiterable.ResultIterable object at 0x7f73a9513110>), ('mom', <pyspark.resultiterable.ResultIterable object at 0x7f73a938ffd0>), ('1', <pyspark.resultiterable.ResultIterable object at 0x7f73a962a3d0>)]

First 3 letter group: thi ['think', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'think', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'things', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'this', 'things', 'this', 'this', 'this']


## Question 5

In [50]:
rdd3_mapped = rdd3.map(lambda x: (x,1))
rdd3_reduced = rdd3_mapped.reduceByKey(lambda x,y : x+y).sortBy(lambda a: -a[1])
print (rdd3_reduced.take(6))

print ('Spark counter:', rdd3_reduced.filter(lambda x : x[0] == 'spark').collect())

[('to', 164), ('in', 143), ('of', 122), ('and', 106), ('we', 103), ('spark', 69)]
Spark counter: [('spark', 69)]


## Question 6

In [66]:
N = rdd3.getNumPartitions()
print ('Before Partitions:', N)
rdd_repart = rdd3.repartition(10)
N = rdd_repart.getNumPartitions()
print ('After Partitions:', N)
print ('First: ', rdd_repart.first())

def func(iterator):
    yield list(iterator).count('spark')

rdd_repart.mapPartitions(func).collect()

Before Partitions: 1
After Partitions: 10
First:  them


[5, 7, 6, 12, 8, 6, 8, 5, 10, 2]

## Question 7

In [82]:
sample1 = rdd3_reduced.sample(withReplacement=True, fraction=0.5)
sample2 = rdd3_reduced.sample(withReplacement=True, fraction=0.5)

print ("Sample 1")
print ('\t First 5:', sample1.take(5))
print ('\t count: ', sample1.count())
print ("Sample 2")
print ('\t First 5:', sample2.take(5))
print ('\t count: ', sample2.count())


Sample 1
	 First 5: [('in', 143), ('of', 122), ('can', 55), ('can', 55), ('it', 40)]
	 count:  495
Sample 2
	 First 5: [('in', 143), ('of', 122), ('and', 106), ('we', 103), ('apache', 52)]
	 count:  462


In [83]:
print ("UNION")
union_1_2 = sample1.union(sample2)
print ('\t First 5:', union_1_2.take(5))
print ('\t count: ', union_1_2.count())

UNION
	 First 5: [('in', 143), ('of', 122), ('can', 55), ('can', 55), ('it', 40)]
	 count:  957


In [84]:
print ("JOIN")
join_1_2 = sample1.join(sample2)
print ('\t First 5:', join_1_2.take(5))
print ('\t count: ', join_1_2.count())


JOIN
	 First 5: [('in', (143, 143)), ('of', (122, 122)), ('an', (16, 16)), ('an', (16, 16)), ('an', (16, 16))]
	 count:  262
